<a href="https://colab.research.google.com/github/Adam-Rao/Cats-and-Dogs/blob/model/CatsAndDogs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [36]:
import _io
import os
import pickle as pkl
from sklearn.utils import shuffle
from typing import List, Tuple
from sklearn.model_selection import train_test_split

In [25]:
CAT_DATA_PATH = "/content/drive/My Drive/Cats and Dogs/PetImages/Cat"
DOG_DATA_PATH = "/content/drive/My Drive/Cats and Dogs/PetImages/Dog"
GENERAL_DATA_PATH = "/content/drive/My Drive/CAD PROJECT/Data"

DATA_TUPLE_TYPE = Tuple[str, str]
DATA_LIST_TYPE = List[DATA_TUPLE_TYPE]

In [20]:
def create_data_list(label: str, path: str) -> DATA_LIST_TYPE:
  """
  Function to create a list of tuples containing file path and label
  Takes in desired label and directory holding files as arguments
  """
  data_list: DATA_LIST_TYPE = []
  for file in os.listdir(path):
    data_tuple: DATA_TUPLE_TYPE = (f"{path}/{file}", label)
    data_list.append(data_tuple)
  
  return data_list

In [21]:
def shuffle_data_list(data_list: DATA_LIST_TYPE) -> List[DATA_LIST_TYPE]:
  """
  Shuffles list containing  data and returns shuffled list
  Takes in list containing data as arguments
  """
  return shuffle(data_list)

In [44]:
def save_data(data_list: DATA_LIST_TYPE, save_path: str) -> None:
  file = open(save_path, "wb")
  pkl.dump(data_list, file)

In [37]:
def load_data(data_path: str) -> _io.BufferedReader:
  return open(data_path, "rb")

In [45]:
cat_data_list = create_data_list("Cat", CAT_DATA_PATH)
dog_data_list = create_data_list("Dog", DOG_DATA_PATH)
general_data_list = cat_data_list + dog_data_list
shuffled_general_data = shuffle_data_list(general_data_list)
save_data(shuffled_general_data, os.path.join(GENERAL_DATA_PATH, "data.pkl"))

In [41]:
while True: pass #keeps codelab from disconnecting

KeyboardInterrupt: ignored